In [ ]:
def describe_dataset(path, num_samples=3):
    # read file
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # stats
    total_chars = len(text)
    total_lines = text.count("\n") + 1
    vocab = sorted(list(set(text)))
    vocab_size = len(vocab)

    # sample lines (non-empty)
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    sample_lines = lines[:num_samples]

    print("=== Dataset Description ===")
    print(f"File: {path}")
    print(f"Total characters: {total_chars}")
    print(f"Total lines: {total_lines}")
    print(f"Vocabulary size: {vocab_size}")
    print(f"Vocabulary: {vocab}")
    print("Sample lines:")
    for i, s in enumerate(sample_lines):
        print(f"{i+1}. {s}")
    print("\n")

describe_dataset("input_childSpeech_trainingSet.txt")
describe_dataset("input_childSpeech_testSet.txt")
describe_dataset("input_shakespeare.txt")



=== Dataset Description ===
File: input_childSpeech_trainingSet.txt
Total characters: 246982
Total lines: 10001
Vocabulary size: 40
Vocabulary: ['\n', ' ', 'A', 'B', 'C', 'D', 'G', 'H', 'I', 'L', 'M', 'N', 'R', 'S', 'U', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
Sample lines:
1. Night night All done
2. Night night I see bird
3. What is that Help me please


=== Dataset Description ===
File: input_childSpeech_testSet.txt
Total characters: 24113
Total lines: 1001
Vocabulary size: 40
Vocabulary: ['\n', ' ', 'A', 'B', 'C', 'D', 'G', 'H', 'I', 'L', 'M', 'N', 'R', 'S', 'U', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
Sample lines:
1. Look airplane I want cookie
2. I love you I found it
3. I see doggy No touch


=== Dataset Description ===
File: input_shakespeare.txt
Total characters: 1115394
Total lines: 40001
Vocabulary size: 65


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 1200
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 192
n_head = 6
n_layer = 2
dropout = 0.2

# ModelA
# n_embd = 96
# n_head = 3
# n_layer = 2

# ModelB
# n_embd = 128
# n_head = 4
# n_layer = 3

torch.manual_seed(1337)
bias_term=False

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input_childSpeech_trainingSet.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=bias_term)
        self.query = nn.Linear(n_embd, head_size, bias=bias_term)
        self.value = nn.Linear(n_embd, head_size, bias=bias_term)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

#generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

SAVE_GPT = 'trained_gpt.pt'

torch.save({
    'model_state_dict': model.state_dict(),
    'vocab': {'stoi': stoi, 'itos': itos},
    'meta': {'n_embd': n_embd, 'n_head': n_head, 'n_layer': n_layer, 'block_size': block_size, 'vocab_size': vocab_size}
}, SAVE_GPT)
print(f"Saved trained model checkpoint to: {SAVE_GPT}")

from google.colab import files
files.download(SAVE_GPT)


0.953512 M parameters
step 0: train loss 3.6608, val loss 3.6603
step 200: train loss 1.1693, val loss 1.1849
step 400: train loss 0.3531, val loss 0.3569
step 600: train loss 0.3448, val loss 0.3495
step 800: train loss 0.3381, val loss 0.3438
step 1000: train loss 0.3338, val loss 0.3404
step 1199: train loss 0.3332, val loss 0.3380

I tired I draw
Where kitty go I hide
All gone Look airplane
Big truck Help me please
All done What is that
Uh oh spill Bath time
I run fast Shoes on
Mama I want play with big red ball outside I wash hands
Yummy apple Daddy play
Where kitty go Yummy apple
Help me please Read book
I did it Help me please
I climb Yummy apple
Read book Come here
Uh oh spilll I make mass hands
I want that Bath tiruck
I sing What is that
I run fast I found it
I wash haide Night night
Read book Daddy play
I want cookie
Saved trained model checkpoint to: trained_gpt.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch, math
from collections import Counter

SAVE_GPT = 'trained_gpt.pt'
TRAIN_FILE = "input_childSpeech_trainingSet.txt"
TEST_FILE  = "input_childSpeech_testSet.txt"

ckpt = torch.load(SAVE_GPT, map_location=device)
stoi = ckpt['vocab']['stoi']
itos = ckpt['vocab']['itos']
meta = ckpt['meta']
block_size = meta['block_size']
vocab_size = meta['vocab_size']

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# encode test data
test_text = open(TEST_FILE, 'r', encoding='utf-8').read()
test_data = torch.tensor(encode(test_text), dtype=torch.long)
if len(test_data) < block_size + 1:
    raise ValueError("Test text too short (need > block_size characters).")

# evaluate test set
def eval_model(model, data_tensor, block_size, device):
    losses = []
    for i in range(0, len(data_tensor) - block_size - 1, block_size):
        x = data_tensor[i:i+block_size].unsqueeze(0).to(device)
        y = data_tensor[i+1:i+block_size+1].unsqueeze(0).to(device)
        _, loss = model(x, y)
        losses.append(loss.item())
    if len(losses) == 0:
        raise ValueError("No evaluation blocks created. Check test length vs block_size.")
    avg_loss = sum(losses) / len(losses)
    return avg_loss, math.exp(avg_loss), len(losses)

model_loss, model_ppl, nblocks = eval_model(model, test_data, block_size, device)

#compute baseline
train_text = open(TRAIN_FILE, 'r', encoding='utf-8').read()

counts = Counter(train_text)
total = sum(counts.values())
eps = 1e-12


def uniform_random_baseline(vocab_size):
    loss = math.log(vocab_size)
    ppl = vocab_size
    return loss, ppl

uni_loss, uni_ppl = uniform_random_baseline(vocab_size)

print(f"ModelC loss = {model_loss:.4f}, ppl = {model_ppl:.2f}")
print(f"Baseline loss = {uni_loss:.4f}, ppl = {uni_ppl:.2f}")


ModelC loss = 0.3456, ppl = 1.41
Baseline loss = 3.6889, ppl = 40.00


In [ ]:
import torch, math
from collections import Counter

SAVE_GPT = 'trained_gpt.pt'
TRAIN_FILE = "input_childSpeech_trainingSet.txt"
SHK_FILE = "input_shakespeare.txt"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = globals().get('block_size', 256)


# --- load checkpoint ---
ckpt = torch.load(SAVE_GPT, map_location=device)
itos = ckpt['vocab']['itos']
stoi = ckpt['vocab']['stoi']

# --- build model ---
model = GPTLanguageModel().to(device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

# --- read Shakespeare and ignore unseen characters ---
text = open(SHK_FILE, 'r', encoding='utf-8').read()
filtered = [c for c in text if c in stoi]   # ignore unknown chars
enc = [stoi[c] for c in filtered]

print(f"Original Shakespeare tokens: {len(text)}")
print(f"Tokens after filtering unknown chars: {len(enc)}")
print(f"Ignored {len(text) - len(enc)} unseen characters.")

data_tensor = torch.tensor(enc, dtype=torch.long)

# --- evaluate model ---
@torch.no_grad()
def eval_model(model, data_tensor, block_size, device):
    losses = []
    for i in range(0, len(data_tensor) - block_size - 1, block_size):
        x = data_tensor[i:i+block_size].unsqueeze(0).to(device)
        y = data_tensor[i+1:i+block_size+1].unsqueeze(0).to(device)
        _, loss = model(x, y)
        losses.append(loss.item())
    avg = sum(losses)/len(losses)
    return avg, math.exp(avg), len(losses)

model_loss, model_ppl, nblocks = eval_model(model, data_tensor, block_size, device)

# --- compute baseline ---
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

baseline_loss, baseline_ppl = uniform_random_baseline(vocab_size)

print(f"\nModel  -> loss={model_loss:.4f}, ppl={model_ppl:.2f}, blocks={nblocks}")
print(f"Baseline -> loss={baseline_loss:.4f}, ppl={baseline_ppl:.2f}")


Original Shakespeare tokens: 1115394
Tokens after filtering unknown chars: 1034258
Ignored 81136 unseen characters.

Model  -> loss=6.5838, ppl=723.28, blocks=4040
Baseline -> loss=4.1744, ppl=65.00
